<a href="https://colab.research.google.com/github/simon-bouchard/freecodecamp-machine-learning-models/blob/main/fcc-ham-or-spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from imblearn.over_sampling import ADASYN

import numpy as np
import matplotlib.pyplot as plt


print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.5/642.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.9 MB/s eta 0:00:00
2.19.0-dev20250128


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-01-28 20:28:36--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.05s   

2025-01-28 20:28:37 (7.42 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-01-28 20:28:37--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
#get data
train_data = pd.read_csv('train-data.tsv', sep='\t', header=None)
valid_data = pd.read_csv('valid-data.tsv', sep='\t', header=None)

train_labels = train_data[0]
train_texts = train_data[1]

valid_labels = valid_data[0]
valid_texts = valid_data[1]

train_labels.shape

(4179,)

In [8]:
#Preprocessing

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
valid_labels = label_encoder.transform(valid_labels)

tokenizer = Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(train_texts)

num_words = len(tokenizer.word_index) + 1
max_len = max(max(len(text.split()) for text in train_texts),
              max(len(text.split()) for text in valid_texts))

train_sequences = tokenizer.texts_to_sequences(train_texts)
train_sequences = pad_sequences(train_sequences, padding='post', maxlen=max_len)

valid_sequences = tokenizer.texts_to_sequences(valid_texts)
valid_sequences = pad_sequences(valid_sequences, padding='post', maxlen=max_len)

In [9]:
#model
inputs = Input(shape=(max_len,))
x = Embedding(input_dim=num_words, output_dim=32, input_length=max_len)(inputs)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Bidirectional(LSTM(32))(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 171)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, 171, 32)             │         246,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 171, 128)            │          49,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 64)                  │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 337,153 (1.29 MB)

 Trainable params: 337,153 (1.29 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
!nvidia-smi
print("GPU available:", tf.config.list_physical_devices('GPU'))


/bin/bash: line 1: nvidia-smi: command not found
GPU available: []


In [10]:
#Train model
#Set class weight (there are more ham than spam)(not necessary)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

class_weights = dict(enumerate(class_weights))
print(f"Class Weights: {class_weights}")

history = model.fit(
    train_sequences,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(valid_sequences, valid_labels),
    class_weight=class_weights
    )

model.save('spam_ham_model.keras')

Class Weights: {0: 0.5773694390715667, 1: 3.73125}
Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 62s 413ms/step - accuracy: 0.7853 - loss: 0.4372 - val_accuracy: 0.9813 - val_loss: 0.1117
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 79s 393ms/step - accuracy: 0.7065 - loss: 0.5187 - val_accuracy: 0.9289 - val_loss: 0.1922
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 85s 417ms/step - accuracy: 0.9482 - loss: 0.1638 - val_accuracy: 0.9145 - val_loss: 0.2287
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 84s 433ms/step - accuracy: 0.9726 - loss: 0.0985 - val_accuracy: 0.9813 - val_loss: 0.0697
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 79s 413ms/step - accuracy: 0.9764 - loss: 0.0992 - val_accuracy: 0.9856 - val_loss: 0.0672
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 81s 407ms/step - accuracy: 0.9898 - loss: 0.0534 - val_accuracy: 0.9842 - val_loss: 0.0726
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 82s 408ms/step - accuracy: 0.9921 - loss: 0.0600 - val_accuracy: 0.9820 - val_loss: 0.0917
Epoch 8/10
131/131 ━━━━━━━━━━━━━

In [11]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

#Uncomment if you don't want to run the previous cells
#model = load_model('spam_ham_model.keras')

def predict_message(pred_text):

  sequence = tokenizer.texts_to_sequences([pred_text])
  sequence = pad_sequences(sequence, padding='post', maxlen=max_len)

  predictions = model.predict(sequence)

  prediction = [predictions[0][0]]

  prediction.append('ham' if predictions[0][0] <= 0.5 else 'spam')

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 693ms/step
[0.0018118859, 'ham']


In [12]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
You passed the challenge. Great job!
